In [45]:
from utils import get_grouped_df

# df = get_grouped_df()
# df.head()

In [46]:
# Charger les données
df = get_grouped_df()
df.head()

/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,nom_acteur,nom_compagnie,nom_genre
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,"John Leguizamo, Jon Favreau, Sofía Vergara, Em...","Altus Media, Fairview Entertainment, Prescience",Comédie
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,"Scarlett Johansson, Jillian Bell, Ilana Glazer...","Paulilu Productions, Sony Pictures, Columbia P...","Drame, Comédie"
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,"Zooey Deschanel, Joseph Gordon-Levitt, Chloë G...","Fox Searchlight Pictures, Watermark Pictures, ...","Comédie, Drame, Romance"
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,"Nanou Garcia, Gilles Lellouche, Valérie Lemerc...","SCOPE Pictures, Rectangle Productions",Comédie
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,"Cliff Curtis, Steven Strait, Nathanael Baring,...","Centropolis Entertainment, Legendary Pictures,...","Aventure, Action, Drame, Fantastique"


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from azureml.core import Workspace, Experiment, Run
from utils import get_grouped_df
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Charger les données
df = get_grouped_df()

# Assurez-vous d'avoir le titre du film
df_title = df['title'].copy()

# Séparer les listes d'acteurs, de compagnies, et de genres en plusieurs colonnes
df['nom_acteur'] = df['nom_acteur'].str.split(', ')
df['nom_compagnie'] = df['nom_compagnie'].str.split(', ')
df['nom_genre'] = df['nom_genre'].str.split(', ')

# Créer un binarizer
mlb = MultiLabelBinarizer()

# Transformer les colonnes
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_acteur')),
                          columns=[f'actor_{cls}' for cls in mlb.classes_],
                          index=df.index))

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_compagnie')),
                          columns=[f'company_{cls}' for cls in mlb.classes_],
                          index=df.index))

df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('nom_genre')),
                          columns=[f'genre_{cls}' for cls in mlb.classes_],
                          index=df.index))

# On remplit les valeurs manquantes par 0
df_t = df.fillna(0)

df_t

/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,genre_Fantastique,genre_Guerre,genre_Histoire,genre_Horreur,genre_Musique,genre_Mystère,genre_Romance,genre_Science-Fiction,genre_Thriller,genre_Western
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
df_t

,title,entries,date,realisateur,budget,titre_non_modifie,film_id,actor_50 Cent,actor_A$AP Rocky,actor_A.J. Cook,...,genre_Fantastique,genre_Guerre,genre_Histoire,genre_Horreur,genre_Musique,genre_Mystère,genre_Romance,genre_Science-Fiction,genre_Thriller,genre_Western
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2741,zerotheorem,40523,2014-06-25,Terry Gilliam,8500000,Zero Theorem,1779,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2742,zookeeper,73107,2011-08-17,Frank Coraci,80000000,Zookeeper,576,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2743,zoolander2,46701,2016-03-02,Ben Stiller,50000000,Zoolander 2,1223,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2744,zootopie,1556396,2016-02-17,Byron Howard,150000000,Zootopie,1931,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder , RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor


In [50]:
def add_new_features(df):
    """Ajout de nouvelles fonctionnalités au DataFrame"""
    df['moy_entries_real'] = df.groupby('realisateur')['entries'].transform('mean')
    df['len_title_chars'] = df['titre_non_modifie'].apply(lambda x: len(x))
    df['len_title_words'] = df['titre_non_modifie'].apply(lambda x: len(x.split()))
    df['nb_film_real'] = df.groupby('realisateur')['titre_non_modifie'].transform('count')
    df['avg_budget_per_film'] = df.groupby('realisateur')['budget'].transform('mean')
    
    # Vous pouvez ajouter plus de caractéristiques ici en fonction de votre compréhension du problème et des données disponibles.
    
    return df

def preprocess_inputs(df):
    """Prétraitement des entrées du DataFrame"""
    df = df.drop(['title', 'date', 'titre_non_modifie', 'film_id'], axis=1)
    y = df['entries']
    X = df.drop('entries', axis=1)
    return X, y


def build_model():
    """Construction du pipeline de modèle"""
    # Transformation polynomiale
    poly_transformer = Pipeline(steps=[
        ('scaler', RobustScaler()),
        ('poly', PolynomialFeatures(degree=2))
    ])

    # Transformation OneHot
    cat_transformer = OneHotEncoder(handle_unknown='ignore')

    # Assemblage des transformations
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', poly_transformer, ['budget', 'moy_entries_real', 'len_title', 'nb_film_real' , 'avg_budget_per_film']),
            ('cat', cat_transformer, ['realisateur'])],
        remainder='passthrough')

    # Création de pipeline pour le modèle RandomForestRegressor
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestRegressor(random_state=42))
    ])
    
    return model

def train_model(model, X_train, y_train):
    """Formation du modèle"""
    model.fit(X_train, y_train)
    return model

In [57]:
from azureml.core import Workspace, Experiment
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import joblib
import datetime

# Connectez-vous à votre espace de travail Azure ML
ws = Workspace.from_config()

# Créez une nouvelle expérience
exp = Experiment(workspace=ws, name='my-experiment')

# Commencez une nouvelle run
run = exp.start_logging()

# Application des fonctions
df = add_new_features(df_t)
X, y = preprocess_inputs(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construction du modèle
model = build_model()

# Définition des paramètres de la grille
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 5, 10]
}

# Création du GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Formation du modèle
grid_search.fit(X_train, y_train)

# Meilleur modèle
best_model = grid_search.best_estimator_

# Prédisez avec le meilleur modèle et calculez l'erreur quadratique moyenne
y_pred = best_model.predict(X_test)
y_pred = y_pred.astype(int)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = best_model.score(X_test, y_test)
rmse = np.sqrt(mse)

mse = mse.astype(int)
mae = mae.astype(int)
rmse = rmse.astype(int)

# Loggez les métriques
run.log('mse', mse)
run.log('mae', mae)
run.log('r2', r2)
run.log('rmse', rmse)

# Timestamp pour rendre le nom du modèle unique
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f"model_{timestamp}.pkl"

# Enregistrez le meilleur modèle dans le répertoire de sortie
joblib.dump(best_model, f'outputs/{model_name}')

# Terminez la run
run.complete()


In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prédictions du modèle
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

# Convertir les prédictions en entier
train_preds = train_preds.astype(int)
test_preds = test_preds.astype(int)

# Calcul des métriques
train_rmse = sqrt(mean_squared_error(y_train, train_preds))
test_rmse = sqrt(mean_squared_error(y_test, test_preds))

train_mae = mean_absolute_error(y_train, train_preds)
test_mae = mean_absolute_error(y_test, test_preds)

train_r2 = r2_score(y_train, train_preds)
test_r2 = r2_score(y_test, test_preds)

# Affichage des métriques
print(f"Train RMSE: {train_rmse}, Test RMSE: {test_rmse}")
print(f"Train MAE: {train_mae}, Test MAE: {test_mae}")
print(f"Train R2: {train_r2}, Test R2: {test_r2}")

Train RMSE: 91886.38630333, Test RMSE: 230360.30452526477
Train MAE: 44243.167122040075, Test MAE: 121522.43090909091
Train R2: 0.9581017645050743, Test R2: 0.7022810212796462


In [ ]:
# On crée un DataFrame pour les prédictions de test
df_test = pd.DataFrame({'Title': df_title.loc[X_test.index], 'Actual': y_test, 'Predicted': test_preds})

# voir le actual le plus haut
df_test.sort_values(by=['Actual'], ascending=False).head(30)

,Title,Actual,Predicted
841,harrypotteretlacoupedefeu,3088796,1944125
1480,leseigneurdesanneauxleretourduroi,2935984,1974505
212,avatar,2925087,2324914
840,harrypotteretlachambredessecrets,2728766,1724249
2018,piratesdescaraibesjusquauboutdumonde,2709377,1560147
2234,shrek2,2238286,1910931
1133,lagedeglace2,2171810,920126
2366,taxi4,2010736,1914741
218,avengersleredultron,1926049,2148308
1501,lesindestructibles2,1840492,1225973
